In [39]:
import pandas as pd
import numpy as np
import glob
import os

In [40]:
def get_important_sentences(df):
    
    THRESHOLD=0.78
    MAX_SENTENCE_NUM=2
    index_list = []
    
    columns = df.columns[2:].tolist()
    columns = [i for i in columns if not "市場予想" in i]
#     sampledict = dict.fromkeys(columns, )
    for column in columns:
        valuelist=[]
        # 最大値とそのインデックス
        maxvalue =df[column].max() 
        maxidx = df[column].idxmax()
        if maxvalue >= 0.7:
            valuelist.append(tuple([maxvalue,maxidx]))
        
        # 閾値以上の値とそのインデックス 
        values=df[column][df[column]>THRESHOLD].values.tolist()
        indexes=df[df[column]>THRESHOLD].index.tolist()
        for v,i in zip(values,indexes):
                valuelist.append(tuple([v,i])) 

        #　重複を除く
        valuelist = list(set(valuelist))

        # スコア降順に並べて最大センテンス数だけ取得
        if len(valuelist)>MAX_SENTENCE_NUM:
            valuelist.sort(key = lambda x: x[0], reverse=True)
            valuelist=valuelist[:MAX_SENTENCE_NUM]

#         sampledict[column]=valuelist
        index_list+=([i[1] for i in valuelist])
    
    important_sentences = df.iloc[index_list]["sentence_list"].tolist()
    return important_sentences, index_list, columns

In [52]:
csv= pd.read_csv('data/data_for_T5_increased.csv')
csv = csv[["file","company","summary"]]


input_csv= pd.read_csv('increased_Tanshin.csv')
youyakus = input_csv[["file","summary"]].values.tolist()

# キーをファイル名、値を要約文の。区切りリストにした辞書を作成
youyaku_sentence_dic = {}
for y in youyakus:
    if y[0] not in youyaku_sentence_dic:
        youyaku_sentence_dic[y[0]] = []
    youyaku_sentence_dic[y[0]].append(y[1])

In [53]:
files = glob.glob("/Users/takahashimichika/Documents/KessanTanshin/BERTscore_precision/*")
for file in files:
    filename = os.path.basename(file)[13:-4]+"pdf"
    df = pd.read_excel(file)
    sentence, index , columns= get_important_sentences(df)
    csv.loc[csv["file"]==filename, ["importants"]] = "".join(sentence)
#     csv.loc[csv["file"]==filename, ["summary"]] ="".join(youyaku_sentence_dic[filename])
#     csv.loc[csv["file"]==filename, ["summary"]] ="".join([i for i in youyaku_sentence_dic[filename][1:] if not"市場予想" in i])

In [54]:
for i, row in csv.iterrows():
#     row["summary"] = row["company"][:-6].strip()+"。" + row["summary"]
    row["importants"] = row["company"][:-6].strip() +"。"+ row["importants"]

In [55]:
csv[5:20]

,file,company,summary,importants
5,kessan33-3.pdf,セコム (9735),セコム、営業利益4.2%減 セキュリティーは回復途上もBPO・ICT事業が下支え。2021年...,セコム。2当期は事業所向け・家庭向けのセントラライズドシステム（オンライン・セキュリティシス...
6,ir20210512_784.pdf,資生堂 (4911),資生堂、1-3月期営業利益67.6％増 事業譲渡に伴い通期営業利益予想は下方修正。2021年...,資生堂。 2021年12月期の通期連結業績予想に関しては、既存ビジネスでの売上高・利益につい...
7,tanshin210803.pdf,ダイキン工業 (6367),ダイキン工業、営業利益実績は前年比2倍に 通期見通しを最高益予想に上方修正。4-6月期の営業...,ダイキン工業。当社グループは、『うるさらX（エックス）』をはじめとする換気機能付き機種の訴求...
8,financialaffairs85.pdf,スズキ (7269),スズキ、営業利益実績は市場予想を上回る 未定としていた通期見通しを開示。4-6月期の営業利益...,スズキ。② 二輪事業新型ハヤブサの投入効果や事業体質改善の進捗等により、売上高は575億円と...
9,2021_tanshin.pdf,アサヒグループホールディングス (2502),アサヒグループ、事業利益は大幅増益を確保 オセアニアなどの国際事業がけん引。4-6月期の事業...,アサヒグループホールディングス。 東南アジア事業については、マレーシアで、『WONDA』...
10,12517741208799999999_d15331e6-5393-4993-b0b0-f...,ダイフク (6383),ダイフク、営業利益実績は市場予想並みの水準 通期見通しは据え置き。4-6月期の営業利益実績は...,ダイフク。このような経済・事業環境のもと、当社グループの受注は、新型コロナウイルス感染症の影...
11,sbkk_financial_report_20210511.pdf,ソフトバンク (9434),ソフトバンク、21年3月期は全セグメントで増益確保も市場予想は下回る。2021年3月期の営業...,"ソフトバンク。(b) 営業利益当期の営業利益は、全セグメントで増益し、前期比59,045百万..."
12,t202106_4Q.pdf,朝日インテック (7747),朝日インテック、2021年6月期の営業利益実績は市場予想並みの水準で着地。2021年6月期の...,朝日インテック。そのような中、当社グループの当連結会計年度における売上高は、新型コロナウイル...
13,fr_20_4q.pdf,ミスミグループ本社 (9962),ミスミグループ本社、FA事業堅調でコンセンサス予想を上回る。2021年3月期については、主力...,ミスミグループ本社。①ＦＡ事業ＦＡ事業は、中国が年間を通して好調に推移したことに加え、期中よ...
14,Neg7.pdf,日本通運 (9062),日本通運、営業利益実績は市場予想を下回る 通期見通しは売上高が上方修正。4-6月期の営業利益...,日本通運。こうした経営環境のもと、当社グループの当第1四半期連結累計期間の業績は、航空運送を...


In [56]:
csv.to_csv("data_for_T5_1202_v3.csv",index=False,encoding="utf-8-sig")

In [8]:
old_csv= pd.read_csv('data/data_for_T5_increased.csv')

In [17]:
old_csv
for i, row in old_csv.iterrows():
    print(row["important_texts"].split("。"))
    print(len(row["important_texts"].split("。"))-1)


['ヤマハ', '当連結会計年度の売上収益は、新型コロナウイルスの感染拡大に伴う影響等により、為替のマイナス影響23億円を含め、前期に対し415億97百万円（10.0%）減少の3,726億30百万円となりました', '事業利益は、為替のマイナス影響６億円を含め、前期に対し56億41百万円（12.2%）減少の407億11百万円となりました', '親会社の所有者に帰属する当期利益は、事業利益の減少に加え、新型コロナウイルスの感染拡大に伴う操業停止損23億18百万円、減損損失35億53百万円の計上もあり、前期に対し80億６百万円（23.1%）減少の266億15百万円となりました', '（楽器事業）アコースティックピアノと電子楽器は、中国での販売が成長軌道に復帰した他、各国の市況が回復基調にあるものの、商品供給不足もあり上期の減収をカバーするまでには至らず、減収となりました', '管楽器は、市況の回復が遅れ減収となりました', 'ギターは、国内や中国で販売を伸ばし増収となりました', '以上により、当事業の売上収益は、前期に対し為替のマイナス影響17億円を含め、303億90百万円（11.3％）減少の2,389億81百万円となりました', '事業利益は、為替のマイナス影響８億円を含め、53億32百万円（14.1％）減少の324億17百万円となりました', '（音響機器事業）オーディオ機器は、ステイホーム需要によりサウンドバー等の販売が伸長しましたが、上期の減収をカバーするまでには至らず、全体では減収となりました', '業務用音響機器は、ライブ市場や設備市場の停滞により減収となりました', '以上により、当事業の売上収益は、前期に対し為替のマイナス影響４億円を含め、105億79百円（9.2％）減少の1,038億13百万円となりました', '事業利益は、為替のプラス影響２億円を含め、15億４百万円（17.5％）減少の70億67百万円となりました', '（その他の事業）電子デバイスは減収となりましたが、自動車用内装部品やFA機器は需要の回復により増収となりました', '以上により、当事業の売上収益は、前期に対し６億26百万円（2.1％）減少の298億36百万円となりました', '事業利益は、前期30百万円に対し、11億95百万円増加の12億25百万円となりました', '②次期の見通

In [22]:
counts = 0
c=0
for i, row in csv.iterrows():
#     print(row["importants"].split("。"))
    sentence_counts = len(row["importants"].split("。"))-1
    counts += sentence_counts
    c+=1
print(counts/c)

5.0


In [27]:
counts = 0
c=0
for i, row in old_csv.iterrows():
#     print(row["importants"].split("。"))
    sentence_counts = len(row["important_texts"].split("。"))-1
    counts += sentence_counts
    c+=1
print(counts/c)

27.95774647887324
